# *Torneio Alfabeta Jogo Focus*
## Introdução à Inteligência Artificial (2025-26)
### Avaliação Contínua 3


## O jogo Focus (Domination)

O [Focus (ou Domination)](https://en.wikipedia.org/wiki/Focus_(board_game)), foi concebido por Sid Sackson e publicado pela primeira vez em 1963. É um jogo estratégico de empilhar peças num tabuleiro especial de forma octogonal.
Cada jogador move pilhas de peças, controlando apenas as que têm a sua cor no topo.

As pilhas deslocam-se na ortogonal tantas casas quanto o seu tamanho, podendo capturar peças adversárias ou guardar as suas na reserva quando se fundem pilhas e o resultado da fusão ultrapassa a altura de 5 peças. **Para facilitar, neste projeto vamos limitar essa altura a três peças e o tabuleiro será mais pequeno do que no jogo original**.


Vence quem conseguir dominar o tabuleiro.

<img src="focus4x4.png" alt="Drawing" style="width: 300px;"/>

### Como funciona o jogo? 

O Focus pode ser jogado por 2 a 4 jogadores, mas neste caso vamos ter apenas dois jogadores. 


#### 1. Início do Jogo
O tabuleiro da nossa versão do Focus tem a forma de um quadrado 4x4 com 4 extensões laterais de duas peças, formando um octógono.
Cada casa válida contém uma única peça, e as peças são dispostas alternadamente entre as cores RED (vermelho) e GREEN (verde), conforme o padrão mostrado na imagem acima.

Cada jogador controla as peças da sua cor:
* O jogador RED começa a partida.
* O jogador GREEN joga em seguida, alternando os turnos.

Durante o jogo, cada jogador mantém dois contadores:
* Reserva – peças próprias retiradas de pilhas demasiado grandes (podem ser recolocadas mais tarde).
* Capturas – peças do adversário removidas do tabuleiro (são eliminadas definitivamente).

#### 2. Jogada

Em cada turno, o jogador cuja vez é de jogar tem duas opções:

(A) Mover uma pilha 
* Pode mover qualquer pilha cujo topo seja da sua cor.
	* A pilha desloca-se em linha reta (horizontal ou vertical) um número de casas igual à quantidade de peças na pilha.
	* Se a casa de destino já contiver uma pilha, ambas são fundidas, colocando a pilha que se está a mover no topo da que está na casa de destino.
	* Se a nova pilha ultrapassar **três peças**, as peças em excesso são removidas, da parte inferior da pilha:
	    * As peças do próprio jogador vão para a sua reserva.
	    * As peças do adversário são capturadas.

(B) Jogar uma peça da reserva
* Em vez de mover uma pilha, o jogador pode colocar uma peça da sua reserva em qualquer casa do tabuleiro (incluindo no topo de uma pilha cuja peça do topo não seja a sua).
* Esta ação é útil para recuperar controlo sobre pilhas dominadas pelo adversário.

NOTA: uma peça única (não empilhada) é por si só um caso particular de uma pilha, cujo topo é a cor dessa mesma peça.

#### 3. Fim do Jogo
O jogo termina quando um dos jogadores não tem mais jogadas possíveis, ou seja:
* não pode mover nenhuma pilha cujo topo seja da sua cor,
* e não tem peças na reserva para colocar.

O **vencedor** é o último jogador que ainda consegue realizar uma jogada válida.

* Como o jogo pode tornar-se de longa duração, vamos assumir que se um limite máximo de jogadas for atingido (por exemplo, 250 jogadas), o jogo termina. Será o vencedor aquele que tiver um maior número de pilhas com a sua cor. Será empate se nesse momento ambos controlarem o mesmo número de pilhas.

## Objetivos do projeto
Pretende-se que, dada uma formulação e implementação do jogo, os grupos de alunos:
<br><br>
Criem um jogador, na forma de uma função de avaliação a ser usada pelo algoritmo alfabeta para qualquer profundidade, par ou ímpar. Aconselhamos que comecem por desenvolver um jogador simples para depois ser progressivamente melhorado, de modo a que tenham um jogador disponível na data de entrega limite. Desenvolvam e comparem o desempenho de vários jogadores, para diferentes limites de profundidade, e depois selecionem o melhor deles para entrega. Cada grupo só pode participar no torneio com um único jogador.
<br><br>
O jogador de cada grupo participará em torneios de todos contra todos, i.e., cada jogador irá jogar vários jogos contra os jogadores de todos os outros grupos, em diferentes níveis de profundidade. O nosso jogador também participará, o <span style="color:orange"> ***Basicus***</span>.

<img src="basicus.png" alt="Drawing" style="width: 200px;"/>
<br>
(imagem gerada pelo GPT-5)



O Basicus sabe as regras do jogo e pouco mais. Quando imagina um tabuleiro que resultaria de uma possível jogada, sabe verificar se esse tabuleiro só tem pilhas cujo topo é da mesma cor, o que significaria uma vitória (se essas pilhas forem dele) ou uma derrota (se forem do adversário). Também sabe que, se não existirem apenas pilhas cujo topo é da sua cor, a *maximização* desse número também é importante. O desempenho do Basicus é a *baseline* que devem ultrapassar de forma a obterem uma nota superior a 10 valores.


## Formulação do Jogo Focus em Python

Vamos descrever de modo sumário como está modelado o jogo Focus, através das classes `EstadoFocus` e `JogoFocus` que estão em `focus.py`.

### O estado do jogo
Na classe `EstadoFocus` temos cinco atributos principais:
* `to_move`: Indica quem é o próximo a jogar (`'RED'` se for o jogador das peças vermelhas, `'GREEN'` se for o das peças verdes).
* `board`: Uma representação do tabuleiro sob a forma de um dicionário que guarda a informação sobre a localização das pilhas no tabuleiro. As chaves são as posições $(linha,coluna)$ das casas ocupadas (sendo a primeira linha a de cima e a primeira coluna a da esquerda) e os valores correspondem a listas com valores das cores dos jogadores, ordenadas da base para o topo. As casas livres não são representadas no board. 
* `reserve`: Indica quantas peças cada jogador tem na sua reserva. A representação é feita através de um dicionário cujas chaves são os jogadores (`'RED'` or `'GREEN'`) e os valores o número de peças correspondentes.
* `captured`: Indica quantas peças cada capturou do adversário. A representação é feita através de um dicionário cujas chaves são os jogadores (`'RED'` or `'GREEN'`) e os valores o número de peças correspondentes.
* `n_jogadas`: Indica quantas jogadas foram feitas desde o início do jogo. Serve para determinar o final do jogo que esteja a ser longo (depois de 250 jogadas contam-se o número de pilhas controladas pelos jogadores e vence o que controlar mais pilhas, ou, se ambos tiverem o mesmo número de pilhas controladas, há um empate).

Temos também os métodos: 

* **`valid_positions()`** (*static*): Função que devolve o conjunto de posições válidas do tabuleiro do Jogo Focus (de acordo com a imagem do tabuleiro acima).

* **`all_positions()`**: Devolve todas as posições possíveis do tabuleiro, sob a forma de lista de coordenadas `(x, y)`.

* **`top_piece(pos)`**: Devolve a cor da peça que está no topo da pilha na posição dada ou `None` no caso da casa se encontrar vazia.

* **`is_valid_position(pos)`**: Verifica se a posição faz parte do tabuleiro.

* **`possible_moves()`**: Devolve uma lista de tuplos com todas as ações possíveis para o jogador que está a jogar. As ações podem ser de dois tipos:
    - `((x, y), dir)` que representa a ação de mover uma pilha na posição `(x, y)` na direção `dir`. `dir` pode tomar os valores `'up'` (cima), `'down'` (baixo), `'left'` (esquerda) ou `'right'` (direita);
    - `('reserve', (x, y))` que representa a ação de colocar no tabuleiro uma peça da reserva do jogador. A peça pode ser colocada em qualquer lugar, mesmo em cima de uma pilha que não seja controlada pelo jogador.

* **`next_state(action)`**: Dada uma jogada (`action`), devolve o **novo estado do jogo** resultante.  
  - Se a jogada for de movimentação `((x, y), dir)`, a peça avança na direção `dir` tantas casas quantas peças estiverem na pilha.  
  - Se a jogada for de colocação de uma peça da reserva `('reserve', (x, y))`, a peça é colocada no topo da pilha da posição indicada.  
  Retorna um novo objeto `EstadoFocus` com o tabuleiro, a reserva e captura atualizados, o jogador seguinte e o número de jogadas incrementado.

* **`winner()`**: Verifica se algum jogador venceu.  
  Devolve `'RED'` ou `'GREEN'` se um dos jogadores tiver vencido o jogo, i.e., o oponente não tem jogadas possíveis. Caso contrário, devolve `None`.

* **`other()`**: Devolve o oponente do jogador `to_move`.

* **`who_dominate()`**: Devolve o jogador que domina todas as pilhas do tabuleiro, ou `None` caso nenhum domine todas.

* **`dominate_piles(player)`**: Devolve o número de pilhas que têm o topo da cor do jogador `player`.

* **`dominate_pieces(player)`**: Devolve o número total de peças controladas pelo jogador `player`, ou seja, todas as peças em pilhas cujo topo é do jogador.

* **`display()`** : Mostra o **tabuleiro do jogo**. Cada posição é representada pelos caracteres `[  ]`. Entre eles são apresentadas as peças da base para o topo, i.e., o primeiro elemento da é a peça da base. Cada peça é representada por um carácter (`R` para *Red* e `G` para *Green*). As casas vazias são representadas vazias. Inclui também índices de linhas e colunas para facilitar a leitura.

Vejamos agora alguns exemplos, após importarmos `focus.py`.

In [2]:
from focus import *

Criemos o estado inicial standard e observemos os seus atributos:

In [3]:
tabuleiro_inicial = criar_tabuleiro_inicial()
estado_inicial = EstadoFocus(to_move='RED', board=tabuleiro_inicial, reserve={'RED':0,'GREEN':0},
                           captured={'RED':0,'GREEN':0}, n_jogadas=0)
print("Próximo jogador:",estado_inicial.to_move)
print("Tabuleiro:",estado_inicial.board)
print("Peças Vermelhas na Reserva:",estado_inicial.reserve['RED'])
print("Peças Verdes na Reserva:",estado_inicial.reserve['GREEN'])
print("Peças Vermelhas Capturadas:",estado_inicial.reserve['RED'])
print("Peças Verdes Capturadas:",estado_inicial.reserve['GREEN'])
print("No.Jogadas:",estado_inicial.n_jogadas)

Próximo jogador: RED
Tabuleiro: {(0, 0): ['GREEN'], (0, 1): ['RED'], (0, 2): ['GREEN'], (0, 3): ['RED'], (1, 0): ['GREEN'], (1, 1): ['RED'], (1, 2): ['GREEN'], (1, 3): ['RED'], (2, 0): ['RED'], (2, 1): ['GREEN'], (2, 2): ['RED'], (2, 3): ['GREEN'], (3, 0): ['RED'], (3, 1): ['GREEN'], (3, 2): ['RED'], (3, 3): ['GREEN']}
Peças Vermelhas na Reserva: 0
Peças Verdes na Reserva: 0
Peças Vermelhas Capturadas: 0
Peças Verdes Capturadas: 0
No.Jogadas: 0


Agora observemos o tabuleiro em modo textual: 

In [ ]:
estado_inicial.display()

Vamos perguntar se temos um vencedor (obviamente que não):

In [ ]:
print(estado_inicial.winner())

Vamos aplicar uma jogada e visualizar o tabuleiro resultante:

In [ ]:
estado_inicial.next_state(((1,3), 'up')).display()

### A classe `JogoFocus`, subclasse de `Game`
A classe `JogoFocus` (em `focus.py`) é uma subclasse de `Game` (em `jogos.py`). 

O construtor gera o estado inicial do jogo com um tabuleiro resultado da função `criar_tabuleiro_inicial()` (ver imagem no início do enunciado), e o jogador 'RED' é o primeiro a jogar.

Temos também os métodos habituais:
* `actions`: Devolve a lista de ações possíveis (jogadas possíveis) para um determinado estado.
* `result`: Devolve o estado que resulta da aplicação de uma ação (jogada) a um outro estado.
* `terminal_test`: Verifica se o jogo acabou ou não.
* `utility`: Devolve +1 se ganhou o jogador, -1 se ganhou o adversário, 0 se empataram.
* `display`: Apresenta em modo de texto o estado do jogo, incluindo o tabuleiro e outras informações.

### Jogadores
Em `jogos.py` temos vários tipos de jogadores e funções para correr jogos.

#### O jogador aleatório
O jogador aleatório escolhe ao acaso uma das jogadas possíveis.
Vamos ver um jogo entre dois jogadores aleatórios:

In [4]:
from focus import *
from jogos import *
jogo = JogoFocus()
jogo.jogar(random_player,random_player)


   Jogadas feitas: 0
   Tabuleiro atual:
-1 |                  [   ]  [   ]              
 0 |           [  G]  [  G]  [  R]  [  R]       
 1 |    [   ]  [  R]  [  R]  [  G]  [  G]  [   ]
 2 |    [   ]  [  G]  [  G]  [  R]  [  R]  [   ]
 3 |           [  R]  [  R]  [  G]  [  G]       
 4 |                  [   ]  [   ]              
        _____  _____  _____  _____  _____  _____
         -1      0      1      2      3      4

  Próximo jogador: RED

  Peças na reserva: 0   (GREEN: 0)
  Peças capturadas: 0   (GREEN: 0)
  Pilhas dominadas: 8   (GREEN: 8)

   Jogadas feitas: 1
   Tabuleiro atual:
-1 |                  [   ]  [   ]              
 0 |           [  G]  [  G]  [  R]  [  R]       
 1 |    [   ]  [   ]  [ RR]  [  G]  [  G]  [   ]
 2 |    [   ]  [  G]  [  G]  [  R]  [  R]  [   ]
 3 |           [  R]  [  R]  [  G]  [  G]       
 4 |                  [   ]  [   ]              
        _____  _____  _____  _____  _____  _____
         -1      0      1      2      3      4

  Pró

-1

O score final é +1(-1) se RED ganhou(perdeu), ou 0 se empataram.
Se quisermos correr um jogo vendo só os scores finais, podemos usar a função `jogar()` em modo não verboso:

In [5]:
jogo=JogoFocus()
jogo.jogar(random_player,random_player,verbose=False)

1

#### O jogador Alfabeta com profundidade limitada
No nosso torneio, os jogadores vão sempre usar o algoritmo Alfabeta. No jogo Focus, não é viável desenvolver a árvore até ao fim, por isso vamos usar o Alfabeta com profundidade limitada (função `alphabeta_cutoff_search` definida em `jogos.py`). Esta função recebe um estado, um jogo, a profundidade de procura, e uma função de avaliação. É a função de avaliação de cada jogador que vai determinar o seu desempenho.

Vamos equipar o jogador aleatório com uma função de avaliação. Ele continuará a escolher jogadas ao acaso, mas conseguirá reconhecer a situação em que o próximo jogador poderá ganhar o jogo (porque pode colocar/mover uma peça e dominar todas as pilhas). Quando é detetado o final iminente do jogo, a função de avaliação deve devolver `infinity`/`-infinity` para maximixar o número de cortes que a função `alphabeta_cutoff_search` faz.

In [6]:
def func_gameover(estado,jogador) :
    clone=copy.deepcopy(estado) #boa prática de programação, para não arriscarem estragar o estado
    winner = clone.winner()
    if winner != None:
        return infinity if winner==jogador else -infinity
    return 0 #em qualquer outra situação que não seja vitória ou derrota

def jogador_random_plus_1(jogo,estado) :
    return alphabeta_cutoff_search(estado,jogo,1,eval_fn=func_gameover)
    

Vejamos a vantagem que traz esta função de avaliação. Reparem que o `jogador_random_plus_1` joga com uma profundidade de procura 1. Quer isto dizer que a procura não vai além da jogada imediata; nem sequer olha para qualquer possível jogada do adversário. Vamos fazer 10 jogos entre o `jogador_random_plus_1` e o jogador aleatório que não reconhece o final do jogo. <br><br>
Reportamos os scores resultantes de cada jogo, e no final a soma de todos os scores, que nos dá a pontuação do primeiro jogador, o `jogador_random_plus_1`. Esta pontuação será positiva(negativa) se o primeiro(segundo) jogador ganhou a maioria dos jogos, e será 0 se ambos os jogadores ganharem o mesmo número de jogos (ou se empatarem todos os jogos).

In [ ]:
pontuacao=0
for i in range(10):
    resultado = jogo.jogar(jogador_random_plus_1,random_player,verbose=False)
    print(resultado)
    pontuacao += resultado
print('Pontuação do jogador_random_plus_1:',pontuacao)

Este jogador parece ser um bocadinho melhor do que o totalmente aleatório. Deverá ser ainda melhor com profundidades maiores. Vejamos os resultados finais obtido em 10 jogos a profundidades 1 a 5 (podem correr várias vezes, reparando como o tempo de execução aumenta com a profundidade):

In [7]:
def jogador_random_plus_p(jogo,estado) :
    return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_gameover)
    
# for p in range(1,6):
#     def jogador_random_plus_p(jogo,estado) :
#         return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_gameover)
    
#     print('Profundidade', p)
#     quem_ganhou=0
#     for i in range(10):
#         resultado = jogo.jogar(jogador_random_plus_p,random_player,verbose=False)
#         #print(resultado)
#         quem_ganhou += resultado
#     print(quem_ganhou)


### O Basicus
Vamos agora definir a função de avaliação do nosso jogador <span style="color:orange"> ***Basicus***</span>, chamada `func_basicus`. Como dito acima, o Basicus sabe reconhecer o final do jogo e sabe é importante dominar o máximo de pilhas.

In [8]:
def func_basicus(estado,jogador) :
    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    my_pilhas = clone.dominate_piles(jogador)
    opponent = 'GREEN' if jogador == 'RED' else 'RED'
    opp_pilhas = clone.dominate_piles(opponent)
    return my_pilhas - opp_pilhas


Vamos verificar como é que esta função de avaliação avalia alguns estados: 

In [ ]:
board_1 = {
    (0, 0): ['RED'],
    (1, 0): ['GREEN'],
    (0, 1): ['GREEN'],
    (1, 1): ['RED'],
}

board_2 = {
    (0, 0): ['RED', 'RED'],
    (1, 0): ['RED'],
    (0, 1): ['RED'],
    (1, 1): ['RED'],
}

board_3 = {
    (0, 0): ['RED'],
    (1, 0): ['RED', 'GREEN', 'RED'],   
    (0, 1): ['GREEN'],
    (1, 1): ['RED'],
}

board_4 = {
    (0, 0): ['GREEN'],
    (1, 0): ['GREEN', 'RED', 'GREEN'],  
    (0, 1): ['RED'],
    (1, 1): ['GREEN'],
}

#para este efeito só interessa inicializar o coerentemente o tabuleiro
est1=EstadoFocus('RED',board_1, {'RED':0, 'GREEN':0}, {'RED':0, 'GREEN':0}, 10) 
est2=EstadoFocus('RED',board_2,{'RED':0, 'GREEN':0}, {'RED':0, 'GREEN':0}, 10)
est3=EstadoFocus('RED',board_3,{'RED':0, 'GREEN':0}, {'RED':0, 'GREEN':0}, 10)
est4=EstadoFocus('RED',board_4,{'RED':0, 'GREEN':0}, {'RED':0, 'GREEN':0}, 10)

jogo=JogoFocus()

# Aqui ninguem ganhou e ambos dominam o mesmo número pilhas
est1.display()
print('Avaliação segundo',est1.to_move,': ',func_basicus(est1,est1.to_move))
print('Avaliação segundo',est1.other(),': ',func_basicus(est1,est1.other()),'\n\n')

# Aqui o RED ganhou
est2.display()
print('Avaliação segundo',est2.to_move,': ',func_basicus(est2,est2.to_move))
print('Avaliação segundo',est2.other(),': ',func_basicus(est2,est2.other()),'\n\n')

# Aqui o RED domina mais pilhas do que o GREEN
est3.display()
print('Avaliação segundo',est3.to_move,': ',func_basicus(est3,est3.to_move))
print('Avaliação segundo',est3.other(),': ',func_basicus(est3,est3.other()),'\n\n')

# Aqui o GREEN domina mais pilhas do que o RED
est4.display()
print('Avaliação segundo',est4.to_move,': ',func_basicus(est4,est4.to_move))
print('Avaliação segundo',est4.other(),': ',func_basicus(est4,est4.other()),'\n\n')


Vamos definir o jogador <span style="color:orange"> ***Basicus***</span> e ver qual o seu desempenho ao jogar com o jogador random_plus com profundidades de procura 1, 2, 3 e 4. Fazemos só 10 jogos com cada profundidade, para não demorar muito. Mostramos também o resultado de cada jogo, para além da pontuação final.

In [9]:
jogo=JogoFocus()

def jogador_random_plus_p(jogo,estado) :
    return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_gameover)

def jogador_basicus_p(jogo,estado) :
    return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_basicus)
    

In [ ]:
for p in range(1,5):
    def jogador_random_plus_p(jogo,estado) :
        return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_gameover)
    def jogador_basicus_p(jogo,estado) :
        return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_basicus)
    
    print('\nProfundidade:',p)
    pontuacao=0
    for i in range(10):
        resultado = jogo.jogar(jogador_basicus_p,jogador_random_plus_p,verbose=False)
        # print(resultado)
        pontuacao += resultado
    print('Pontuação do Basicus:',pontuacao)

Curiosamente, a estratégia do Basicus parece funcionar melhor a profundidades mais baixas e às vezes chega mesmo a perder contra o jogador aleatório. Podemos também correr um jogo com o utilizador humano. Atenção: o jogador humano deve indicar sempre uma das jogadas possíveis, caso contrário, o jogo não é válido. Tentem ganhar ao Basicus!

In [ ]:
jogo=JogoFocus()
p=1
# jogo.jogar(query_player,jogador_basicus_p) # descomentar para jogar

### Funções de avaliação compostas
Obviamente que o Basicus, mesmo sendo melhor (em média) do que o jogador aleatório, não é lá grande jogador. Maximar o número de pilhas dominadas não lhe serve de muito se o adversário tiver liberdade para cobrir as suas pilhas na jogada seguinte. Para obterem um jogador melhor, poderão ter que combinar vários critérios na mesma função de avaliação! 

### N Pares de jogos
Vejamos agora uma função que realiza $N$ pares de jogos entre dois jogadores - $N$ jogos com um deles a jogar primeiro, $N$ jogos com o outro a jogar primeiro. Devolve um tuplo com 4 elementos: o número de vitórias de cada um e de empates quando um deles joga primeiro; o número de vitórias de cada um e de empates quando o outro joga primeiro; o número total de vitórias de cada um e de empates; e finalmente o score de cada um. A pontuação depende da tabela de `scores`, que neste caso indica que a vitória vale $3$, a derrota $0$ e o empate $1$. É a mesma escala que iremos utilizar no torneio.

In [10]:
def jogador_basicus_1(jogo,estado) :
    return alphabeta_cutoff_search(estado,jogo,1,eval_fn=func_basicus)

In [11]:
# Dicionário global de pontuação
SCORES = {'Vitoria': 3, 'Empate': 1}

def traduz_pontos(tabela):
    """
    Converte uma tabela de resultados (vitórias, empates) em pontuação.
    """
    empates = tabela['Empate']
    return {
        jogador: SCORES['Vitoria'] * vitorias + SCORES['Empate'] * empates
        for jogador, vitorias in tabela.items() if jogador != 'Empate'
    }

def joga_n_pares(jogo, n, jog1, jog2):
    """
    Executa n pares de jogos entre jog1 e jog2 (ida e volta).
    Retorna:
      - tabelaPrim: resultados da primeira sequência (jog1 vs jog2)
      - tabelaSeg: resultados da segunda sequência (jog2 vs jog1)
      - tabela: soma das duas
      - tabela_pontos: pontuação traduzida
    """
    name_jog1, name_jog2 = jog1.__name__, jog2.__name__
    inicial = {name_jog1: 0, name_jog2: 0, 'Empate': 0}
    tabela_prim, tabela_seg = inicial.copy(), inicial.copy()

    for _ in range(n):
        # Jogo 1: jog1 vs jog2
        vencedor = jogo.jogar(jog1, jog2, verbose=False)
        if vencedor > 0:
            tabela_prim[name_jog1] += 1
        elif vencedor < 0:
            tabela_prim[name_jog2] += 1
        else:
            tabela_prim['Empate'] += 1

        # Jogo 2: jog2 vs jog1 (inverso)
        vencedor = jogo.jogar(jog2, jog1, verbose=False)
        if vencedor > 0:
            tabela_seg[name_jog2] += 1
        elif vencedor < 0:
            tabela_seg[name_jog1] += 1
        else:
            tabela_seg['Empate'] += 1

    tabela_total = {
        chave: tabela_prim[chave] + tabela_seg[chave]
        for chave in tabela_prim
    }

    return tabela_prim, tabela_seg, tabela_total, traduz_pontos(tabela_total)

Façamos 5 pares de jogos entre o `jogador_basicus_1` e o `jogador_random_plus_1` e vejamos a pontuação final:

In [12]:
jogo=JogoFocus()
joga_n_pares(jogo, 5, jogador_basicus_1, jogador_random_plus_1)

({'jogador_basicus_1': 5, 'jogador_random_plus_1': 0, 'Empate': 0},
 {'jogador_basicus_1': 5, 'jogador_random_plus_1': 0, 'Empate': 0},
 {'jogador_basicus_1': 10, 'jogador_random_plus_1': 0, 'Empate': 0},
 {'jogador_basicus_1': 30, 'jogador_random_plus_1': 0})

### Torneio entre vários jogadores
Eis a função que realiza um torneio entre vários jogadores, em que cada um realiza um número $N$ de jogos contra todos os outros como primeiro jogador, e $N$ como segundo jogador:

In [61]:
def incorpora(tabela, nova_tabela):
    """
    Soma os valores de 'nova_tabela' à tabela principal.
    """
    for jogador, pontos in nova_tabela.items():
        tabela[jogador] = tabela.get(jogador, 0) + pontos


def torneio(n, jogadores, jogo):
    """
    Executa um torneio round-robin entre os jogadores fornecidos.

    Parâmetros:
        n (int): número de pares de jogos (ida e volta) entre cada dupla.
        jogadores (list): lista de funções/jogadores.
        jogo (obj): jogo (ex: JogoFocus).

    Retorna:
        dict: tabela final ordenada por pontuação.
    """
    tabela = {}

    for i, jog1 in enumerate(jogadores[:-1]):
        for jog2 in jogadores[i+1:]:
            print(f"{jog1.__name__} VS {jog2.__name__}")
            _, _, _, tabela_parcial = joga_n_pares(jogo, n, jog1, jog2)
            incorpora(tabela, tabela_parcial)

    tabela_final = dict(sorted(tabela.items(), key=lambda x: x[1], reverse=True))

    print("\n🏆 Classificação Final 🏆")
    for pos, (jogador, pontos) in enumerate(tabela_final.items(), start=1):
        print(f"{pos:>2}. {jogador:<15} {pontos:>4} pontos")

    return tabela_final

Fazemos agora um torneio com três dos jogadores que definimos até agora, com 5 pares de jogos entre eles.<br> Os resultados são apresentados do melhor para o pior jogador:

In [20]:
def jogador_grupo28_p(jogo,estado) :
    return alphabeta_cutoff_search(estado,jogo,1,eval_fn=func_28)

In [67]:
modifiers_list = [[44, 32, 19,  8, 14, 38, 33, 4, 1,  8, 15, 43,  4, 23, 46],
[43, 24, 20,  8, 16, 38, 22,  2,  5,  8, 10,  2, 13, 23, 25],
[43, 41, 36, 15, 33, 47, 22, 21, 11, 21, 48, 17, 24, 34,  3],
[44, 33, 12,  8, 14, 38, 42,  2, 22, 35, 20, 49,  2, 48, 14],
[43, 32,  6,  8, 12, 21, 12, 10,  1, 30,  5, 49, 24, 38,  4],
[44, 32, 19,  8, 14, 38, 33, 18, 11,  8, 15, 43,  8, 13, 43],
[43, 24, 20,  8, 16, 38, 32, 20,  5,  8, 35, 41, 41,  2,  3],
[44, 33, 12,  8, 14, 38, 33, 23, 11, 30, 30, 43,  4, 19, 46]]



def test_modifiers():    
        
        for p in range(1, 3): 
            print("DEPTH:", p)

            def jogador_grupo28_BEA(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_bea)
            def jogador_grupo28_1(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_1)
            def jogador_grupo28_2(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_2)
            def jogador_grupo28_3(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_3)
            def jogador_grupo28_4(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_4)
            def jogador_grupo28_5(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_5)
            def jogador_grupo28_6(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_6)
            def jogador_grupo28_7(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_7)
            def jogador_grupo28_8(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_28_8)
            def jogador_basicus_p(jogo,estado) :
                return alphabeta_cutoff_search(estado,jogo,p,eval_fn=func_basicus)
    
            torneio(2,[jogador_basicus_p, jogador_grupo28_1, jogador_grupo28_2, jogador_grupo28_3, jogador_grupo28_5], JogoFocus())

test_modifiers()

DEPTH: 1
jogador_basicus_p VS jogador_grupo28_1
jogador_basicus_p VS jogador_grupo28_2
jogador_basicus_p VS jogador_grupo28_3
jogador_basicus_p VS jogador_grupo28_5
jogador_grupo28_1 VS jogador_grupo28_2
jogador_grupo28_1 VS jogador_grupo28_3
jogador_grupo28_1 VS jogador_grupo28_5
jogador_grupo28_2 VS jogador_grupo28_3
jogador_grupo28_2 VS jogador_grupo28_5
jogador_grupo28_3 VS jogador_grupo28_5

🏆 Classificação Final 🏆
 1. jogador_grupo28_5   40 pontos
 2. jogador_grupo28_3   22 pontos
 3. jogador_grupo28_1   20 pontos
 4. jogador_grupo28_2   20 pontos
 5. jogador_basicus_p    8 pontos
DEPTH: 2
jogador_basicus_p VS jogador_grupo28_1
jogador_basicus_p VS jogador_grupo28_2
jogador_basicus_p VS jogador_grupo28_3
jogador_basicus_p VS jogador_grupo28_5
jogador_grupo28_1 VS jogador_grupo28_2
jogador_grupo28_1 VS jogador_grupo28_3
jogador_grupo28_1 VS jogador_grupo28_5
jogador_grupo28_2 VS jogador_grupo28_3
jogador_grupo28_2 VS jogador_grupo28_5
jogador_grupo28_3 VS jogador_grupo28_5

🏆 Clas

E pronto, agora divirtam-se a derrotar o Basicus!

<img src="basicus.png" alt="Drawing" style="width: 200px;"/>
<br>
(imagem gerada pelo GPT-5)

## Material a entregar
Devem entregar um ficheiro **Focus_proj_grupoXX.py** (em que XX é o número do grupo registado no moodle), com o código da vossa função de avaliação e todas as funções auxiliares que criarem.

<span style="color:red"> **Não alterem**</span> `focus.py`, `utils.py` nem `jogos.py` e **não devem submetê-los!**
<br><span style="color:red">**Não redefinam**</span> funções com o mesmo nome das já existentes nestes ficheiros.

A função de avaliação do vosso jogador deverá ter assinatura **`func_XX(estado,jogador)`** (substituindo XX pelo número do grupo) e deve devolver o valor estimado do `estado` na perspectiva do `jogador`. Não precisam de definir o vosso jogador, pois seremos nós a defini-lo com diferentes profundidades durante a execução do campeonato.

**Todas** as vossas funções devem ter o sufixo **_XX** (substituindo XX pelo número do grupo), para que não se partilhe nem se sobreponha código durante a avaliação.

## Avaliação
A nota do vosso projecto depende da pontuação final obtida no torneio *múltiplo*. 
   
**Torneios**: Cada jogador irá participar em torneios contra todos os outros, fazendo pelo menos dois jogos (um como primeiro jogador, outro como segundo jogador) contra cada um dos outros jogadores em cada torneio. Faremos torneios a diferentes profundidades, pelo menos profundidade 3 e profundidade 4. Se o tempo de execução o permitir, faremos mais jogos por torneio, e mais torneios a profundidades maiores. Caso o tempo de execução se torne mesmo problemático, poderemos ter de baixar o número máximo de jogadas para 100.

**Pontuação**: Nos torneios, vamos sempre incluir o nosso jogador <span style="color:orange"> ***Basicus***</span>. A pontuação obtida por cada grupo em cada torneio será a *soma dos pontos obtidos em todos os jogos* (em que cada vitória vale 3, cada empate vale 1, e cada derrota vale 0). A pontuação final será a soma das pontuações obtidas em todos os torneios.

**Nota**: Quem obtiver a mesma pontuação final que o <span style="color:orange"> ***Basicus***</span>, terá 10 valores. Quem tiver uma pontuação inferior ao Basicus terá nota menor do que 10. A nota será resultado da aplicação de uma função linear baseada na pontuação obtida, tanto acima do Basicus como para baixo. O grupo que ficar em primeiro lugar no torneio (naturalmente, acima do Basicus), terá 20 valores.

**Clones**: Qualquer jogador que use a mesma estratégia do <span style="color:orange"> ***Basicus***</span> ou de outro jogador definido no enunciado será desclassificado e a sua nota final será 0. Jogadores que sejam cópias uns dos outros também serão desclassificados. 

**Timeout**: Se um jogador ultrapassar o tempo limite para uma jogada, será também desclassificado. Usaremos bom senso para definir o que é o tempo considerado como limite: será o tempo acima do qual a execução do torneio fica comprometida.

**Ficheiros**: Se um grupo entregar ficheiros que não seguem as regras definidas acima (nomeadamente, no que respeita aos sufixos nos nomes das funções), sofrerá uma penalização na nota proporcional ao tempo necessário para resolver a situação. No limite, não será possível a sua participação no torneio e a nota será 0.

In [57]:
import copy
from jogos import infinity

# --- (Versão 7) ---

# W_PILHAS_28: Lógica do Basicus.
W_PILHAS_28 = 10

# --- BÓNUS DE DESEMPATE ---

# W_CAPTURA_28: Capturar é quase tão bom como ganhar uma pilha.
W_CAPTURA_28 = 8

# W_RESERVA_28: Ter reservas é muito bom.
W_RESERVA_28 = 6

# W_ALTURA_H3_28: Um bónus extra por ter uma pilha H3.
# (Valor total da pilha H3 = 10 + 5 = 15)
W_ALTURA_H3_28 = 5

# W_ALTURA_H2_28: Um bónus extra por ter uma pilha H2.
# (Valor total da pilha H2 = 10 + 3 = 13)
W_ALTURA_H2_28 = 3


def func_28_bea(estado, jogador):
    """
    Função de avaliação (Versão 7).
    Usa a lógica do Basicus (contar pilhas) como base (peso 10)
    e bónus (captura, reserva, altura) que são suficientemente
    grandes para serem usados como desempate.
    """

    # --- 1. Verificação de Estado Terminal ---
    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    if winner is not None:
        return infinity if winner == jogador else -infinity

    # --- 2. Preparação ---
    opponent = clone.other()
    score = 0.0

    # --- 3. Heurísticas Ponderadas ---

    # Bónus A: Vantagem de Reserva
    my_reserva = clone.reserve[jogador]
    opp_reserva = clone.reserve[opponent]
    score += (my_reserva - opp_reserva) * W_RESERVA_28

    # Bónus B: Vantagem Material
    my_capturadas = clone.captured[jogador]
    opp_capturadas = clone.captured[opponent]
    score += (my_capturadas - opp_capturadas) * W_CAPTURA_28

    # --- Estratégia Principal ---
    
    my_h3_stacks = 0
    my_h2_stacks = 0
    opp_h3_stacks = 0
    opp_h2_stacks = 0
    my_pilhas_total = 0
    opp_pilhas_total = 0

    for stack in clone.board.values():
        if not stack:
            continue

        top_piece = stack[-1]
        height = len(stack)

        if top_piece == jogador:
            # Estratégia Principal
            my_pilhas_total += 1
            
            # Bónus C: Altura
            if height == 3:
                my_h3_stacks += 1
            elif height == 2:
                my_h2_stacks += 1
        
        else: # top_piece == opponent
            opp_pilhas_total += 1
            if height == 3:
                opp_h3_stacks += 1
            elif height == 2:
                opp_h2_stacks += 1

    # Pontuação (peso 10)
    score += (my_pilhas_total - opp_pilhas_total) * W_PILHAS_28
    
    # Aplicar Bónus de Altura
    score += (my_h3_stacks - opp_h3_stacks) * W_ALTURA_H3_28
    score += (my_h2_stacks - opp_h2_stacks) * W_ALTURA_H2_28

    # --- 4. Retorno da Pontuação ---
    return score

In [46]:
from focus import EstadoFocus, JogoFocus
from typing import Callable


import copy
import math

infinity = float('inf')


# funcao usada como funcao de avaliacao num algoritmo alphabeta com cutoff
# estado eh o estado atual do jogo
# jogador eh a string como a cor do jogador "RED" ou "GREEN"
# retorna o valor avaliado deste estado
# Objetivo eh escrever uma funcao que maximiza a pontuacao no torneio
                                        
def func_28_1(estado: EstadoFocus, jogador: str) -> float:
    EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT = 1.0, 1.0, 1.0, 1.0, 1.0

    mods = [44, 32, 19,  8, 14, 38, 33, 4, 1,  8, 15, 43,  4, 23, 46]

    EARLYRES = mods[0]/100
    EARLYDOM = mods[1]/100
    EARLYPOS = mods[2]/100
    EARLYTOWER = mods[3]/100
    EARLYCAPT = mods[4]/100
    
    MIDRES = mods[5]/100
    MIDDOM = mods[6]/100
    MIDPOS = mods[7]/100
    MIDTOWER = mods[8]/100
    MIDCAPT = mods[9]/100

    LATERES = mods[10]/100
    LATEDOM = mods[11]/100
    LATEPOS = mods[12]/100
    LATETOWER = mods[13]/100
    LATECAPT = mods[14]/100 
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    """
    
    """
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """

    """
    STRATEGY
    - dominate the other player by being the only player that can make a move -> control all the stacks while opponent has no pieces in reserve
    - whole stacks are moved the height of stack
    - pieces are removed from the bottom of the stack if it is taller than 3.
    - having more pieces in reserve in endgame is very important
    """

    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"
    stage = game_stage_28(estado, jogador, opponent)

    capt_val = captured_value_28(estado, jogador, opponent)

    if estado.n_jogadas >= 225:
        return capt_val
    
    if stage > 6:
        # EARLY GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT
    elif stage > 3:
        # MID GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT
    else:
        # LATE GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT

    res_val = reserve_value_28(estado, jogador, opponent)
    dom_val = domination_value_28(estado, jogador, opponent)
    pos_val = position_value_28(estado, jogador, opponent)
    tower_val = tower_value_28(estado, jogador, opponent)

    result = (res_val * res_weight) + (dom_val * dom_weight) + (pos_val * pos_weight) + (tower_val * tower_weight) + (capt_val * capt_weight)
    # print(result)
    return result



def game_stage_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    """
    Function that returns an estimate of the percentage of the game that has passed according to
    the number of pieces still in play
    """
    pieces_left = 8 - estado.captured[opponent]
    return pieces_left




def reserve_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the player's reserve
    """
    res_plyr = estado.reserve[jogador]
    res_opp = estado.reserve[opponent]
    
    return res_plyr - res_opp




def domination_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    return plyr_doms - opp_doms





def position_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the board's position
    """

    # Having opponents pieces on the very bottom of stacks is good and better if you control the stack
    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_base_counter = 0
    for pos in positions:
        opp_base_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            opp_base_counter += 2 * opp_base_pieces
        else:
            opp_base_counter += opp_base_pieces

    return opp_base_counter




def tower_value_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_tower_height = 0

    for pos in positions: 
        if estado.top_piece(pos) == opponent:
            opp_tower_height += len(pos)

    opp_tower_height_avg = opp_tower_height / opp_doms

    # Discourage short enemy towers, because they have very high mobility
    # 3 is ideal average enemy height, 1 is worst

    opp_tower_value = (4 - opp_tower_height_avg) 

    return opp_tower_value




def captured_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the pieces captured 
    """
    capt_plyr = estado.captured[opponent]
    capt_opp = estado.captured[jogador]

    return capt_opp - capt_plyr

In [47]:
from focus import EstadoFocus, JogoFocus
from typing import Callable


import copy
import math

infinity = float('inf')


# funcao usada como funcao de avaliacao num algoritmo alphabeta com cutoff
# estado eh o estado atual do jogo
# jogador eh a string como a cor do jogador "RED" ou "GREEN"
# retorna o valor avaliado deste estado
# Objetivo eh escrever uma funcao que maximiza a pontuacao no torneio
                                        
def func_28_2(estado: EstadoFocus, jogador: str) -> float:
    EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT = 1.0, 1.0, 1.0, 1.0, 1.0

    mods = [43, 24, 20, 8, 16, 38, 22, 2, 5, 8, 10, 2, 13, 23, 25]

    EARLYRES = mods[0]/100
    EARLYDOM = mods[1]/100
    EARLYPOS = mods[2]/100
    EARLYTOWER = mods[3]/100
    EARLYCAPT = mods[4]/100
    
    MIDRES = mods[5]/100
    MIDDOM = mods[6]/100
    MIDPOS = mods[7]/100
    MIDTOWER = mods[8]/100
    MIDCAPT = mods[9]/100

    LATERES = mods[10]/100
    LATEDOM = mods[11]/100
    LATEPOS = mods[12]/100
    LATETOWER = mods[13]/100
    LATECAPT = mods[14]/100 
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    """
    
    """
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """

    """
    STRATEGY
    - dominate the other player by being the only player that can make a move -> control all the stacks while opponent has no pieces in reserve
    - whole stacks are moved the height of stack
    - pieces are removed from the bottom of the stack if it is taller than 3.
    - having more pieces in reserve in endgame is very important
    """

    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"
    stage = game_stage_28(estado, jogador, opponent)

    capt_val = captured_value_28(estado, jogador, opponent)

    if estado.n_jogadas >= 225:
        return capt_val
    
    if stage > 6:
        # EARLY GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT
    elif stage > 3:
        # MID GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT
    else:
        # LATE GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT

    res_val = reserve_value_28(estado, jogador, opponent)
    dom_val = domination_value_28(estado, jogador, opponent)
    pos_val = position_value_28(estado, jogador, opponent)
    tower_val = tower_value_28(estado, jogador, opponent)

    result = (res_val * res_weight) + (dom_val * dom_weight) + (pos_val * pos_weight) + (tower_val * tower_weight) + (capt_val * capt_weight)
    # print(result)
    return result



def game_stage_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    """
    Function that returns an estimate of the percentage of the game that has passed according to
    the number of pieces still in play
    """
    pieces_left = 8 - estado.captured[opponent]
    return pieces_left




def reserve_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the player's reserve
    """
    res_plyr = estado.reserve[jogador]
    res_opp = estado.reserve[opponent]
    
    return res_plyr - res_opp




def domination_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    return plyr_doms - opp_doms





def position_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the board's position
    """

    # Having opponents pieces on the very bottom of stacks is good and better if you control the stack
    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_base_counter = 0
    for pos in positions:
        opp_base_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            opp_base_counter += 2 * opp_base_pieces
        else:
            opp_base_counter += opp_base_pieces

    return opp_base_counter




def tower_value_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_tower_height = 0

    for pos in positions: 
        if estado.top_piece(pos) == opponent:
            opp_tower_height += len(pos)

    opp_tower_height_avg = opp_tower_height / opp_doms

    # Discourage short enemy towers, because they have very high mobility
    # 3 is ideal average enemy height, 1 is worst

    opp_tower_value = (4 - opp_tower_height_avg) 

    return opp_tower_value




def captured_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the pieces captured 
    """
    capt_plyr = estado.captured[opponent]
    capt_opp = estado.captured[jogador]

    return capt_opp - capt_plyr

In [48]:
from focus import EstadoFocus, JogoFocus
from typing import Callable


import copy
import math

infinity = float('inf')


# funcao usada como funcao de avaliacao num algoritmo alphabeta com cutoff
# estado eh o estado atual do jogo
# jogador eh a string como a cor do jogador "RED" ou "GREEN"
# retorna o valor avaliado deste estado
# Objetivo eh escrever uma funcao que maximiza a pontuacao no torneio
                                        
def func_28_3(estado: EstadoFocus, jogador: str) -> float:
    EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT = 1.0, 1.0, 1.0, 1.0, 1.0

    mods = [43, 41, 36, 15, 33, 47, 22, 21, 11, 21, 48, 17, 24, 34, 3]

    EARLYRES = mods[0]/100
    EARLYDOM = mods[1]/100
    EARLYPOS = mods[2]/100
    EARLYTOWER = mods[3]/100
    EARLYCAPT = mods[4]/100
    
    MIDRES = mods[5]/100
    MIDDOM = mods[6]/100
    MIDPOS = mods[7]/100
    MIDTOWER = mods[8]/100
    MIDCAPT = mods[9]/100

    LATERES = mods[10]/100
    LATEDOM = mods[11]/100
    LATEPOS = mods[12]/100
    LATETOWER = mods[13]/100
    LATECAPT = mods[14]/100 
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    """
    
    """
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """

    """
    STRATEGY
    - dominate the other player by being the only player that can make a move -> control all the stacks while opponent has no pieces in reserve
    - whole stacks are moved the height of stack
    - pieces are removed from the bottom of the stack if it is taller than 3.
    - having more pieces in reserve in endgame is very important
    """

    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"
    stage = game_stage_28(estado, jogador, opponent)

    capt_val = captured_value_28(estado, jogador, opponent)

    if estado.n_jogadas >= 225:
        return capt_val
    
    if stage > 6:
        # EARLY GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT
    elif stage > 3:
        # MID GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT
    else:
        # LATE GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT

    res_val = reserve_value_28(estado, jogador, opponent)
    dom_val = domination_value_28(estado, jogador, opponent)
    pos_val = position_value_28(estado, jogador, opponent)
    tower_val = tower_value_28(estado, jogador, opponent)

    result = (res_val * res_weight) + (dom_val * dom_weight) + (pos_val * pos_weight) + (tower_val * tower_weight) + (capt_val * capt_weight)
    # print(result)
    return result



def game_stage_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    """
    Function that returns an estimate of the percentage of the game that has passed according to
    the number of pieces still in play
    """
    pieces_left = 8 - estado.captured[opponent]
    return pieces_left




def reserve_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the player's reserve
    """
    res_plyr = estado.reserve[jogador]
    res_opp = estado.reserve[opponent]
    
    return res_plyr - res_opp




def domination_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    return plyr_doms - opp_doms





def position_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the board's position
    """

    # Having opponents pieces on the very bottom of stacks is good and better if you control the stack
    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_base_counter = 0
    for pos in positions:
        opp_base_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            opp_base_counter += 2 * opp_base_pieces
        else:
            opp_base_counter += opp_base_pieces

    return opp_base_counter




def tower_value_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_tower_height = 0

    for pos in positions: 
        if estado.top_piece(pos) == opponent:
            opp_tower_height += len(pos)

    opp_tower_height_avg = opp_tower_height / opp_doms

    # Discourage short enemy towers, because they have very high mobility
    # 3 is ideal average enemy height, 1 is worst

    opp_tower_value = (4 - opp_tower_height_avg) 

    return opp_tower_value




def captured_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the pieces captured 
    """
    capt_plyr = estado.captured[opponent]
    capt_opp = estado.captured[jogador]

    return capt_opp - capt_plyr

In [49]:
from focus import EstadoFocus, JogoFocus
from typing import Callable


import copy
import math

infinity = float('inf')


# funcao usada como funcao de avaliacao num algoritmo alphabeta com cutoff
# estado eh o estado atual do jogo
# jogador eh a string como a cor do jogador "RED" ou "GREEN"
# retorna o valor avaliado deste estado
# Objetivo eh escrever uma funcao que maximiza a pontuacao no torneio
                                        
def func_28_4(estado: EstadoFocus, jogador: str) -> float:
    EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT = 1.0, 1.0, 1.0, 1.0, 1.0

    mods = [44, 33, 12, 8, 14, 38, 42, 2, 22, 35, 20, 49, 2, 48, 14]

    EARLYRES = mods[0]/100
    EARLYDOM = mods[1]/100
    EARLYPOS = mods[2]/100
    EARLYTOWER = mods[3]/100
    EARLYCAPT = mods[4]/100
    
    MIDRES = mods[5]/100
    MIDDOM = mods[6]/100
    MIDPOS = mods[7]/100
    MIDTOWER = mods[8]/100
    MIDCAPT = mods[9]/100

    LATERES = mods[10]/100
    LATEDOM = mods[11]/100
    LATEPOS = mods[12]/100
    LATETOWER = mods[13]/100
    LATECAPT = mods[14]/100 
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    """
    
    """
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """

    """
    STRATEGY
    - dominate the other player by being the only player that can make a move -> control all the stacks while opponent has no pieces in reserve
    - whole stacks are moved the height of stack
    - pieces are removed from the bottom of the stack if it is taller than 3.
    - having more pieces in reserve in endgame is very important
    """

    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"
    stage = game_stage_28(estado, jogador, opponent)

    capt_val = captured_value_28(estado, jogador, opponent)

    if estado.n_jogadas >= 225:
        return capt_val
    
    if stage > 6:
        # EARLY GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT
    elif stage > 3:
        # MID GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT
    else:
        # LATE GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT

    res_val = reserve_value_28(estado, jogador, opponent)
    dom_val = domination_value_28(estado, jogador, opponent)
    pos_val = position_value_28(estado, jogador, opponent)
    tower_val = tower_value_28(estado, jogador, opponent)

    result = (res_val * res_weight) + (dom_val * dom_weight) + (pos_val * pos_weight) + (tower_val * tower_weight) + (capt_val * capt_weight)
    # print(result)
    return result



def game_stage_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    """
    Function that returns an estimate of the percentage of the game that has passed according to
    the number of pieces still in play
    """
    pieces_left = 8 - estado.captured[opponent]
    return pieces_left




def reserve_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the player's reserve
    """
    res_plyr = estado.reserve[jogador]
    res_opp = estado.reserve[opponent]
    
    return res_plyr - res_opp




def domination_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    return plyr_doms - opp_doms





def position_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the board's position
    """

    # Having opponents pieces on the very bottom of stacks is good and better if you control the stack
    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_base_counter = 0
    for pos in positions:
        opp_base_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            opp_base_counter += 2 * opp_base_pieces
        else:
            opp_base_counter += opp_base_pieces

    return opp_base_counter




def tower_value_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_tower_height = 0

    for pos in positions: 
        if estado.top_piece(pos) == opponent:
            opp_tower_height += len(pos)

    opp_tower_height_avg = opp_tower_height / opp_doms

    # Discourage short enemy towers, because they have very high mobility
    # 3 is ideal average enemy height, 1 is worst

    opp_tower_value = (4 - opp_tower_height_avg) 

    return opp_tower_value




def captured_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the pieces captured 
    """
    capt_plyr = estado.captured[opponent]
    capt_opp = estado.captured[jogador]

    return capt_opp - capt_plyr

In [50]:
from focus import EstadoFocus, JogoFocus
from typing import Callable


import copy
import math

infinity = float('inf')


# funcao usada como funcao de avaliacao num algoritmo alphabeta com cutoff
# estado eh o estado atual do jogo
# jogador eh a string como a cor do jogador "RED" ou "GREEN"
# retorna o valor avaliado deste estado
# Objetivo eh escrever uma funcao que maximiza a pontuacao no torneio
                                        
def func_28_5(estado: EstadoFocus, jogador: str) -> float:
    EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT = 1.0, 1.0, 1.0, 1.0, 1.0

    mods = [43, 32, 6, 8, 12, 21, 12, 10, 1, 30, 5, 49, 24, 38, 4]

    EARLYRES = mods[0]/100
    EARLYDOM = mods[1]/100
    EARLYPOS = mods[2]/100
    EARLYTOWER = mods[3]/100
    EARLYCAPT = mods[4]/100
    
    MIDRES = mods[5]/100
    MIDDOM = mods[6]/100
    MIDPOS = mods[7]/100
    MIDTOWER = mods[8]/100
    MIDCAPT = mods[9]/100

    LATERES = mods[10]/100
    LATEDOM = mods[11]/100
    LATEPOS = mods[12]/100
    LATETOWER = mods[13]/100
    LATECAPT = mods[14]/100 
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    """
    
    """
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """

    """
    STRATEGY
    - dominate the other player by being the only player that can make a move -> control all the stacks while opponent has no pieces in reserve
    - whole stacks are moved the height of stack
    - pieces are removed from the bottom of the stack if it is taller than 3.
    - having more pieces in reserve in endgame is very important
    """

    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"
    stage = game_stage_28(estado, jogador, opponent)

    capt_val = captured_value_28(estado, jogador, opponent)

    if estado.n_jogadas >= 225:
        return capt_val
    
    if stage > 6:
        # EARLY GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT
    elif stage > 3:
        # MID GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT
    else:
        # LATE GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT

    res_val = reserve_value_28(estado, jogador, opponent)
    dom_val = domination_value_28(estado, jogador, opponent)
    pos_val = position_value_28(estado, jogador, opponent)
    tower_val = tower_value_28(estado, jogador, opponent)

    result = (res_val * res_weight) + (dom_val * dom_weight) + (pos_val * pos_weight) + (tower_val * tower_weight) + (capt_val * capt_weight)
    # print(result)
    return result



def game_stage_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    """
    Function that returns an estimate of the percentage of the game that has passed according to
    the number of pieces still in play
    """
    pieces_left = 8 - estado.captured[opponent]
    return pieces_left




def reserve_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the player's reserve
    """
    res_plyr = estado.reserve[jogador]
    res_opp = estado.reserve[opponent]
    
    return res_plyr - res_opp




def domination_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    return plyr_doms - opp_doms





def position_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the board's position
    """

    # Having opponents pieces on the very bottom of stacks is good and better if you control the stack
    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_base_counter = 0
    for pos in positions:
        opp_base_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            opp_base_counter += 2 * opp_base_pieces
        else:
            opp_base_counter += opp_base_pieces

    return opp_base_counter




def tower_value_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_tower_height = 0

    for pos in positions: 
        if estado.top_piece(pos) == opponent:
            opp_tower_height += len(pos)

    opp_tower_height_avg = opp_tower_height / opp_doms

    # Discourage short enemy towers, because they have very high mobility
    # 3 is ideal average enemy height, 1 is worst

    opp_tower_value = (4 - opp_tower_height_avg) 

    return opp_tower_value




def captured_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the pieces captured 
    """
    capt_plyr = estado.captured[opponent]
    capt_opp = estado.captured[jogador]

    return capt_opp - capt_plyr

In [51]:
from focus import EstadoFocus, JogoFocus
from typing import Callable


import copy
import math

infinity = float('inf')


# funcao usada como funcao de avaliacao num algoritmo alphabeta com cutoff
# estado eh o estado atual do jogo
# jogador eh a string como a cor do jogador "RED" ou "GREEN"
# retorna o valor avaliado deste estado
# Objetivo eh escrever uma funcao que maximiza a pontuacao no torneio
                                        
def func_28_6(estado: EstadoFocus, jogador: str) -> float:
    EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT = 1.0, 1.0, 1.0, 1.0, 1.0

    mods = [44, 32, 19, 8, 14, 38, 33, 18, 11, 8, 15, 43, 8, 13, 43]

    EARLYRES = mods[0]/100
    EARLYDOM = mods[1]/100
    EARLYPOS = mods[2]/100
    EARLYTOWER = mods[3]/100
    EARLYCAPT = mods[4]/100
    
    MIDRES = mods[5]/100
    MIDDOM = mods[6]/100
    MIDPOS = mods[7]/100
    MIDTOWER = mods[8]/100
    MIDCAPT = mods[9]/100

    LATERES = mods[10]/100
    LATEDOM = mods[11]/100
    LATEPOS = mods[12]/100
    LATETOWER = mods[13]/100
    LATECAPT = mods[14]/100
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    """
    
    """
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """

    """
    STRATEGY
    - dominate the other player by being the only player that can make a move -> control all the stacks while opponent has no pieces in reserve
    - whole stacks are moved the height of stack
    - pieces are removed from the bottom of the stack if it is taller than 3.
    - having more pieces in reserve in endgame is very important
    """

    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"
    stage = game_stage_28(estado, jogador, opponent)

    capt_val = captured_value_28(estado, jogador, opponent)

    if estado.n_jogadas >= 225:
        return capt_val
    
    if stage > 6:
        # EARLY GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT
    elif stage > 3:
        # MID GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT
    else:
        # LATE GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT

    res_val = reserve_value_28(estado, jogador, opponent)
    dom_val = domination_value_28(estado, jogador, opponent)
    pos_val = position_value_28(estado, jogador, opponent)
    tower_val = tower_value_28(estado, jogador, opponent)

    result = (res_val * res_weight) + (dom_val * dom_weight) + (pos_val * pos_weight) + (tower_val * tower_weight) + (capt_val * capt_weight)
    # print(result)
    return result



def game_stage_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    """
    Function that returns an estimate of the percentage of the game that has passed according to
    the number of pieces still in play
    """
    pieces_left = 8 - estado.captured[opponent]
    return pieces_left




def reserve_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the player's reserve
    """
    res_plyr = estado.reserve[jogador]
    res_opp = estado.reserve[opponent]
    
    return res_plyr - res_opp




def domination_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    return plyr_doms - opp_doms





def position_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the board's position
    """

    # Having opponents pieces on the very bottom of stacks is good and better if you control the stack
    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_base_counter = 0
    for pos in positions:
        opp_base_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            opp_base_counter += 2 * opp_base_pieces
        else:
            opp_base_counter += opp_base_pieces

    return opp_base_counter




def tower_value_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_tower_height = 0

    for pos in positions: 
        if estado.top_piece(pos) == opponent:
            opp_tower_height += len(pos)

    opp_tower_height_avg = opp_tower_height / opp_doms

    # Discourage short enemy towers, because they have very high mobility
    # 3 is ideal average enemy height, 1 is worst

    opp_tower_value = (4 - opp_tower_height_avg) 

    return opp_tower_value




def captured_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the pieces captured 
    """
    capt_plyr = estado.captured[opponent]
    capt_opp = estado.captured[jogador]

    return capt_opp - capt_plyr

In [52]:
from focus import EstadoFocus, JogoFocus
from typing import Callable


import copy
import math

infinity = float('inf')


# funcao usada como funcao de avaliacao num algoritmo alphabeta com cutoff
# estado eh o estado atual do jogo
# jogador eh a string como a cor do jogador "RED" ou "GREEN"
# retorna o valor avaliado deste estado
# Objetivo eh escrever uma funcao que maximiza a pontuacao no torneio
                                        
def func_28_7(estado: EstadoFocus, jogador: str) -> float:
    EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT = 1.0, 1.0, 1.0, 1.0, 1.0

    mods = [43, 24, 20, 8, 16, 38, 32, 20, 5, 8, 35, 41, 41, 2, 3]

    EARLYRES = mods[0]/100
    EARLYDOM = mods[1]/100
    EARLYPOS = mods[2]/100
    EARLYTOWER = mods[3]/100
    EARLYCAPT = mods[4]/100
    
    MIDRES = mods[5]/100
    MIDDOM = mods[6]/100
    MIDPOS = mods[7]/100
    MIDTOWER = mods[8]/100
    MIDCAPT = mods[9]/100

    LATERES = mods[10]/100
    LATEDOM = mods[11]/100
    LATEPOS = mods[12]/100
    LATETOWER = mods[13]/100
    LATECAPT = mods[14]/100
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    """
    
    """
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """

    """
    STRATEGY
    - dominate the other player by being the only player that can make a move -> control all the stacks while opponent has no pieces in reserve
    - whole stacks are moved the height of stack
    - pieces are removed from the bottom of the stack if it is taller than 3.
    - having more pieces in reserve in endgame is very important
    """

    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"
    stage = game_stage_28(estado, jogador, opponent)

    capt_val = captured_value_28(estado, jogador, opponent)

    if estado.n_jogadas >= 225:
        return capt_val
    
    if stage > 6:
        # EARLY GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT
    elif stage > 3:
        # MID GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT
    else:
        # LATE GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT

    res_val = reserve_value_28(estado, jogador, opponent)
    dom_val = domination_value_28(estado, jogador, opponent)
    pos_val = position_value_28(estado, jogador, opponent)
    tower_val = tower_value_28(estado, jogador, opponent)

    result = (res_val * res_weight) + (dom_val * dom_weight) + (pos_val * pos_weight) + (tower_val * tower_weight) + (capt_val * capt_weight)
    # print(result)
    return result



def game_stage_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    """
    Function that returns an estimate of the percentage of the game that has passed according to
    the number of pieces still in play
    """
    pieces_left = 8 - estado.captured[opponent]
    return pieces_left




def reserve_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the player's reserve
    """
    res_plyr = estado.reserve[jogador]
    res_opp = estado.reserve[opponent]
    
    return res_plyr - res_opp




def domination_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    return plyr_doms - opp_doms





def position_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the board's position
    """

    # Having opponents pieces on the very bottom of stacks is good and better if you control the stack
    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_base_counter = 0
    for pos in positions:
        opp_base_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            opp_base_counter += 2 * opp_base_pieces
        else:
            opp_base_counter += opp_base_pieces

    return opp_base_counter




def tower_value_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_tower_height = 0

    for pos in positions: 
        if estado.top_piece(pos) == opponent:
            opp_tower_height += len(pos)

    opp_tower_height_avg = opp_tower_height / opp_doms

    # Discourage short enemy towers, because they have very high mobility
    # 3 is ideal average enemy height, 1 is worst

    opp_tower_value = (4 - opp_tower_height_avg) 

    return opp_tower_value




def captured_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the pieces captured 
    """
    capt_plyr = estado.captured[opponent]
    capt_opp = estado.captured[jogador]

    return capt_opp - capt_plyr

In [53]:
from focus import EstadoFocus, JogoFocus
from typing import Callable


import copy
import math

infinity = float('inf')


# funcao usada como funcao de avaliacao num algoritmo alphabeta com cutoff
# estado eh o estado atual do jogo
# jogador eh a string como a cor do jogador "RED" ou "GREEN"
# retorna o valor avaliado deste estado
# Objetivo eh escrever uma funcao que maximiza a pontuacao no torneio
                                        
def func_28_8(estado: EstadoFocus, jogador: str) -> float:
    EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT = 1.0, 1.0, 1.0, 1.0, 1.0
    LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT = 1.0, 1.0, 1.0, 1.0, 1.0

    mods = [44, 33, 12, 8, 14, 38, 33, 23, 11, 30, 30, 43, 4, 19, 46]

    EARLYRES = mods[0]/100
    EARLYDOM = mods[1]/100
    EARLYPOS = mods[2]/100
    EARLYTOWER = mods[3]/100
    EARLYCAPT = mods[4]/100
    
    MIDRES = mods[5]/100
    MIDDOM = mods[6]/100
    MIDPOS = mods[7]/100
    MIDTOWER = mods[8]/100
    MIDCAPT = mods[9]/100

    LATERES = mods[10]/100
    LATEDOM = mods[11]/100
    LATEPOS = mods[12]/100
    LATETOWER = mods[13]/100
    LATECAPT = mods[14]/100
    """
    Estado do jogo Focus:
    - 'board': dicionário {(x,y): [peças, de baixo para cima]}
    - 'reserve': {'RED': n, 'GREEN': n}
    - 'captured': {'RED': n, 'GREEN': n}
    """
    
    """
    Jogador retorna uma jogada/action/move que pode estar na formas:
    - ((x,y), direction) ex ((1,3), "up")   --  move da pos (1,3) para (1, 3 - n ) se a pilha tiver altura n
    ou 
    - ('reserve', (x,y))   -- tira da reserva de pecas e coloca na posicao (x,y)
    """

    """
    STRATEGY
    - dominate the other player by being the only player that can make a move -> control all the stacks while opponent has no pieces in reserve
    - whole stacks are moved the height of stack
    - pieces are removed from the bottom of the stack if it is taller than 3.
    - having more pieces in reserve in endgame is very important
    """


    clone = copy.deepcopy(estado)
    winner = clone.winner()
    
    # --- Caso terminal ---
    if winner is not None:
        return infinity if winner == jogador else -infinity

    opponent = "GREEN" if jogador == "RED" else "RED"
    stage = game_stage_28(estado, jogador, opponent)

    capt_val = captured_value_28(estado, jogador, opponent)

    if estado.n_jogadas >= 225:
        return capt_val
    
    if stage > 6:
        # EARLY GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT
    elif stage > 3:
        # MID GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT
    else:
        # LATE GAME
        res_weight, dom_weight, pos_weight, tower_weight, capt_weight = LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT

    res_val = reserve_value_28(estado, jogador, opponent)
    dom_val = domination_value_28(estado, jogador, opponent)
    pos_val = position_value_28(estado, jogador, opponent)
    tower_val = tower_value_28(estado, jogador, opponent)

    result = (res_val * res_weight) + (dom_val * dom_weight) + (pos_val * pos_weight) + (tower_val * tower_weight) + (capt_val * capt_weight)
    # print(result)
    return result



def game_stage_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    """
    Function that returns an estimate of the percentage of the game that has passed according to
    the number of pieces still in play
    """
    pieces_left = 8 - estado.captured[opponent]
    return pieces_left




def reserve_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the player's reserve
    """
    res_plyr = estado.reserve[jogador]
    res_opp = estado.reserve[opponent]
    
    return res_plyr - res_opp




def domination_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    # Dominating more stacks is good
    plyr_doms = estado.dominate_piles(jogador)
    opp_doms = estado.dominate_piles(opponent)

    return plyr_doms - opp_doms





def position_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the board's position
    """

    # Having opponents pieces on the very bottom of stacks is good and better if you control the stack
    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_base_counter = 0
    for pos in positions:
        opp_base_pieces = len(list(filter(lambda x: x == opponent, pos)))
        if estado.top_piece(pos) == jogador:
            opp_base_counter += 2 * opp_base_pieces
        else:
            opp_base_counter += opp_base_pieces

    return opp_base_counter




def tower_value_28(estado: EstadoFocus, jogador: str, opponent: str) -> float:
    opp_doms = estado.dominate_piles(opponent)

    positions = [pos for pos in estado.board if len(estado.board[pos]) > 1 ]

    opp_tower_height = 0

    for pos in positions: 
        if estado.top_piece(pos) == opponent:
            opp_tower_height += len(pos)

    opp_tower_height_avg = opp_tower_height / opp_doms

    # Discourage short enemy towers, because they have very high mobility
    # 3 is ideal average enemy height, 1 is worst

    opp_tower_value = (4 - opp_tower_height_avg) 

    return opp_tower_value




def captured_value_28(estado: EstadoFocus, jogador: str, opponent) -> float:
    """
    Function that provides a normalized estimate of the value of the pieces captured 
    """
    capt_plyr = estado.captured[opponent]
    capt_opp = estado.captured[jogador]

    return capt_opp - capt_plyr

In [17]:
def run_tournament(input):
    global EARLYRES, EARLYDOM, EARLYPOS, EARLYTOWER, EARLYCAPT, MIDRES, MIDDOM, MIDPOS, MIDTOWER, MIDCAPT, LATERES, LATEDOM, LATEPOS, LATETOWER, LATECAPT
    print(input)
    EARLYRES = input[0]/100
    EARLYDOM = input[1]/100
    EARLYPOS = input[2]/100
    EARLYTOWER = input[3]/100
    EARLYCAPT = input[4]/100
    
    MIDRES = input[5]/100
    MIDDOM = input[6]/100
    MIDPOS = input[7]/100
    MIDTOWER = input[8]/100
    MIDCAPT = input[9]/100

    LATERES = input[10]/100
    LATEDOM = input[11]/100
    LATEPOS = input[12]/100
    LATETOWER = input[13]/100
    LATECAPT = input[14]/100

    counter = 0
   
    num = 120
    for p in range(1,3):
        def jogador_basicus_p(jogo,estado) :
            return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_basicus)

        def jogador_grupo_28(jogo,estado) :
            return alphabeta_cutoff_search_new(estado,jogo,p,eval_fn=func_28)
    
        quem_ganhou=0
    
        for i in range(num):
            # print("PLAYER 28 IS RED")
            resultado = jogo.jogar(jogador_grupo_28,jogador_basicus_p,verbose=False)
            # print("PLAYER 28 IS GREEN")
            resultado1 = -jogo.jogar(jogador_basicus_p,jogador_grupo_28,verbose=False)
        
            quem_ganhou += resultado + resultado1
        print('Profundidade', p, quem_ganhou)
        counter += quem_ganhou
        # print(quem_ganhou)
        
    print((counter / (num * 4))  * 100)
    return counter

In [ ]:
run_tournament([44, 32, 19,  8, 14, 38, 33, 4, 1,  8, 15, 43,  4, 23, 46])

[44, 32, 19, 8, 14, 38, 33, 4, 1, 8, 15, 43, 4, 23, 46]
Profundidade 1 31


KeyboardInterrupt: 